In [1]:
push!(LOAD_PATH,".")

5-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "/home/skh/MultiAgentSensing"
 "."

In [2]:
using Revise
using POMDPs
using POMDPSimulators
using Dates

In [3]:
using MDMA

┌ Info: Precompiling MDMA [top-level]
└ @ Base loading.jl:1423


Test Summary: | Pass  Total
detectTarget  |   30     30
Test Summary:        | Pass  Total
single_robot_planner |    2      2


In [4]:
sensor = ViewConeSensor(pi/2, 3);
horizon = 15
grid = Grid(15,15, horizon);

targets = Vector{Target}(undef, 0);
push!(targets, Target(grid.width-2,grid.height-2,0));
push!(targets, Target(grid.width-3,grid.height-3,0));
push!(targets, Target(grid.width-1,grid.height-3,0));
push!(targets, Target(20,5,0));

move_dist = 3
model = MDMA.SingleRobotMultiTargetViewCoverageProblem(grid, sensor, horizon, targets, move_dist);
# model.target_trajectories[3,:];

In [5]:
policy = MDMA.solve_single_robot(model)


[Iteration 1   ] residual:        Inf | iteration runtime:   3298.532 ms, (       3.3 s total)
[Iteration 2   ] residual:        Inf | iteration runtime:   3039.202 ms, (      6.34 s total)
[Iteration 3   ] residual:        Inf | iteration runtime:   2931.634 ms, (      9.27 s total)
[Iteration 4   ] residual:        Inf | iteration runtime:   2834.989 ms, (      12.1 s total)
[Iteration 5   ] residual:        Inf | iteration runtime:   2835.903 ms, (      14.9 s total)
[Iteration 6   ] residual:        Inf | iteration runtime:   2832.388 ms, (      17.8 s total)
[Iteration 7   ] residual:        Inf | iteration runtime:   2848.145 ms, (      20.6 s total)
[Iteration 8   ] residual:        Inf | iteration runtime:   2874.815 ms, (      23.5 s total)
[Iteration 9   ] residual:        Inf | iteration runtime:   2923.094 ms, (      26.4 s total)
[Iteration 10  ] residual:        Inf | iteration runtime:   2848.498 ms, (      29.3 s total)
[Iteration 11  ] residual:        Inf | iteration 

ValueIterationPolicy:
 MDPState(UAVState(1, 1, :E, false), 1, nothing) -> MDPState(UAVState(3, 1, :NE, false), 2, nothing)
 MDPState(UAVState(1, 2, :E, false), 1, nothing) -> MDPState(UAVState(3, 4, :NE, false), 2, nothing)
 MDPState(UAVState(1, 3, :E, false), 1, nothing) -> MDPState(UAVState(3, 4, :NE, false), 2, nothing)
 MDPState(UAVState(1, 4, :E, false), 1, nothing) -> MDPState(UAVState(3, 4, :NE, false), 2, nothing)
 MDPState(UAVState(1, 5, :E, false), 1, nothing) -> MDPState(UAVState(3, 4, :NE, false), 2, nothing)
 MDPState(UAVState(1, 6, :E, false), 1, nothing) -> MDPState(UAVState(3, 4, :NE, false), 2, nothing)
 MDPState(UAVState(1, 7, :E, false), 1, nothing) -> MDPState(UAVState(3, 5, :NE, false), 2, nothing)
 MDPState(UAVState(1, 8, :E, false), 1, nothing) -> MDPState(UAVState(3, 6, :NE, false), 2, nothing)
 MDPState(UAVState(1, 9, :E, false), 1, nothing) -> MDPState(UAVState(3, 7, :NE, false), 2, nothing)
 MDPState(UAVState(1, 10, :E, false), 1, nothing) -> MDPState(UAVStat

In [ ]:
#MDMA.neighbors(model, MDbwPState(UAVState(10,10,:N)), model.move_dist)


In [6]:
function compute_path(model, policy, state)
    path = Vector{MDPState}(undef, 0)
    push!(path, state)
    for x in 2:model.horizon
        state = action(policy, state);
#         state = MDPState(state);
        push!(path, state);
    end
    path
end


15-element Vector{MDPState}:
 MDPState(UAVState(10, 10, :N, false), 1, nothing)
 MDPState(UAVState(12, 10, :NE, false), 2, nothing)
 MDPState(UAVState(11, 11, :E, false), 3, nothing)
 MDPState(UAVState(12, 9, :NE, false), 4, nothing)
 MDPState(UAVState(11, 10, :E, false), 5, nothing)
 MDPState(UAVState(12, 8, :NE, false), 6, nothing)
 MDPState(UAVState(11, 9, :E, false), 7, nothing)
 MDPState(UAVState(12, 7, :NE, false), 8, nothing)
 MDPState(UAVState(11, 8, :E, false), 9, nothing)
 MDPState(UAVState(12, 6, :NE, false), 10, nothing)
 MDPState(UAVState(11, 7, :E, false), 11, nothing)
 MDPState(UAVState(12, 5, :NE, false), 12, nothing)
 MDPState(UAVState(11, 6, :E, false), 13, nothing)
 MDPState(UAVState(12, 4, :NE, false), 14, nothing)
 MDPState(UAVState(11, 5, :E, false), 15, nothing)

In [7]:
using Cairo

#Pixels per meter
function init_cairo(model,ppm, buf)
    width = ppm*dims(model.grid)[1]+ 2*buf*ppm
    height = ppm*dims(model.grid)[2]+ 2*buf*ppm
    c = CairoRGBSurface(width,height);
    cr = CairoContext(c);
    draw_background(cr, width, height)
    return (c, cr)
end


function draw_background(cr::CairoContext, width, height)
    save(cr);
    set_source_rgb(cr,1,1,1);    # white
    rectangle(cr,0.0,0.0,width,height); # background
    fill(cr);
    restore(cr);
end

function draw_grid(g::Grid, cr::CairoContext, point_size, ppm, buf)
    set_source_rgba(cr, 0, 0, 0, 0.3);
    x,y,z = dims(g)
    print(dims(g))
    for i in 1:x
        for j in 1:y
            arc(cr, i*ppm + buf*ppm, j*ppm+ buf*ppm, point_size, 0, 2*pi);
            fill(cr);
        end
    end    
end

function draw_target(cr::CairoContext, t::Target, ppm, size, buf)
    save(cr)
    set_source_rgba(cr, 0, 0.3, 0.5, 1);
    arc(cr, t.x*ppm + buf*ppm, t.y*ppm + buf*ppm, size, 0, 2*pi);
    fill(cr);
    restore(cr)
end

function draw_targets(cr::CairoContext, targs::Vector{Target}, ppm, size, buf)
    
    for (i,t) in enumerate(targs)
        cfade = i/length(targs)
        set_source_rgba(cr, (cfade), (1-cfade)*0.5, (1-cfade)*1.1, 1/2);
        draw_target(cr, t,ppm, size, buf)
    end
end

function draw_state(cr::CairoContext, state::UAVState, model, ppm, fade, cfade, buf)
    save(cr)
    move_to(cr,state.x*ppm + buf*ppm, state.y*ppm + buf*ppm)
    fov = model.sensor.fov
    radius = model.sensor.cutoff
    draw_arc(cr, radius, state.x, state.y, state.heading, fov, ppm, fade,cfade, buf)
    restore(cr)
end
function draw_arc(cr::CairoContext, radius, x,y, heading,fov,ppm, fade,cfade, buf) 
    ## original example, following here
    xc = x*ppm + buf*ppm;
    yc = y*ppm + buf*ppm;
    radius = radius*ppm;
    angle1 = dirAngle(heading) + (-fov/2);  # angles are specified
    angle2 = dirAngle(heading) + (fov/2);  # in radians

#     set_source_rgba(cr, 0, 0, 0, fade);
    set_source_rgba(cr, (cfade), (1-cfade)*0.5, (1-cfade)*1.1, fade);
    set_line_width(cr, 5.0);
    arc(cr, xc, yc, radius, angle1, angle2);
    fill(cr)
    stroke(cr);

    # draw helping lines
#     set_source_rgba(cr, cfade, 1-cfade, 1-cfade, fade);
    set_line_width(cr, 6.0);

    # Draw center
    arc(cr, xc, yc, 10.0, 0, 2*pi);
    fill(cr);
    
    arc(cr, xc, yc, radius, angle1, angle1);
    line_to(cr, xc, yc);
#     fill(cr)
    
    arc(cr, xc, yc, radius, angle2, angle2);
    line_to(cr, xc, yc);
#     fill(cr)
    close_path(cr);
    set_source_rgba(cr, (cfade), (1-cfade)*0.5, (1-cfade)*1.1, fade/2);
    fill_preserve(cr);
    
    stroke(cr);
end


# draw_state(cr, UAVState(1,1,:S), model, ppm)


## mark picture with current date


draw_arc (generic function with 1 method)

In [8]:

function draw_path(model, ppm, path, cutoff, buf)
    c, cr = init_cairo(model, ppm, buf)

    draw_grid(model.grid, cr, 5, ppm, buf)

    save(cr)
    select_font_face(cr, "Latin Modern Math", Cairo.FONT_SLANT_NORMAL,
                 Cairo.FONT_WEIGHT_NORMAL);

    set_font_size(cr, 80.0);
    set_source_rgba(cr, 0, 0, 0, 1);
    move_to(cr, model.grid.width/2*ppm + -2*ppm, model.grid.height*ppm + 1.6*buf*ppm);
    show_text(cr, "𝐷 = $(model.move_dist)  ε = 6.8E-07");
    restore(cr)
  

    # Draw States
    state=path[1]
    move_to(cr,state.state.x*ppm + buf*ppm, state.state.y*ppm + buf*ppm)
    draw_state(cr, state.state, model, ppm, 0.4, 1/cutoff, buf)
    for (i,state) in enumerate(path[2:cutoff])
        draw_state(cr, state.state, model, ppm, 0.4, i/cutoff, buf)         
    end
    
    # Draw Lines
    state=path[1]
    move_to(cr,state.state.x*ppm + buf*ppm, state.state.y*ppm + buf*ppm)
    for (i,state) in enumerate(path[2:cutoff])
        cfade = i/cutoff
        fade=1
        set_source_rgba(cr, (cfade), (1-cfade)*0.5, (1-cfade)*1.1, fade/2);
#         set_source_rgba(cr, 0.9, 0.6, 0.01, 1);
        set_line_width(cr, 13.0);
        move_to(cr,path[i].state.x*ppm+ buf*ppm, path[i].state.y*ppm + buf*ppm)
        line_to(cr, state.state.x*ppm+buf*ppm, state.state.y*ppm + buf*ppm)
        stroke(cr) 
        targets = model.target_trajectories[i,:]
        draw_targets(cr, targets,ppm, 15, buf)
         
    end
    
    
    write_to_png(c,"move.png");
end

draw_path (generic function with 1 method)

In [12]:
path = compute_path(model, policy, MDPState(UAVState(1,1,:N)))
draw_path(model, 50, path, model.horizon, 4)

(15, 15, 8, 15)

0

In [10]:
#draw_path(model, 50, compute_path(model, policy, MDPState(UAVState(10,10,:E)), model.horizon, 4)